In [7]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

Use ngbem as HOBEM solver 
=============================


As first example how it works, consider $$ \begin{cases} \Delta u &=& 0, \quad \Omega \subset \mathbb R^3\,,\\ u&=& u_0, \quad \Gamma = \partial \Omega\,.\end{cases} $$ 

Consider the following ansatz for the solution $u\in H^1(\Omega)$ of the above bvp (direct ansatz) $$ u(x) = \underbrace{ \int_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, u_1(y)\, \mathrm{d}\sigma_y}_{\displaystyle{ \mathrm{ SL}(u_1) }} - \underbrace{ \int_\Gamma n(y)\cdot \displaystyle{\frac{1}{4\,\pi}\, \frac{x-y}{\| x-y\|^3} } \, u_0(y)\, \mathrm{d}\sigma_y}_{\displaystyle{ \mathrm{DL}(u_0) }}  $$ and solve for the Neumann data $u_1 \in H^{-\frac12}(\Gamma)$ by the boundary element method, i.e., $$ \forall \, v\in H^{-\frac12}(\Gamma): \quad \left\langle \gamma_0 \left(\mathrm{SL}(u_1)\right), v \right\rangle_{-\frac12} = \left\langle u_0, v\right\rangle_{-\frac12} + \left\langle \gamma_0 \left(\mathrm{DL}(u_0)\right), v\right\rangle_{-\frac12} \,. $$

Define the domain $\Omega \subset \mathbb R^3$ and create a mesh:

In [2]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(2)
#mesh = Mesh(unit_cube.GenerateMesh(maxh=1))

Define dirichlet data:

In [3]:
u0 = 1/ sqrt( (x+1)**2 + (y+1)**2 + (z+1)**2 )

Create the finite element spaces for $H^{-\frac12}(\Gamma)$ and $H^{\frac12}(\Gamma)$ according to the given mesh:  

In [4]:
fesL2 = SurfaceL2(mesh, order=3, dual_mapping=True)
u,v = fesL2.TnT()
fesH1 = H1(mesh, order=3)
uH1,vH1 = fesH1.TnT()

In [5]:
K = BilinearForm(trialspace=fesH1, testspace=fesL2, nonassemble=True)
DoubleLayerPotentialOperator(K, intorder=10, leafsize=40, eta=3., eps=1e-10, method="svd")
K += - 0.5 * uH1 * v.Trace() * ds(bonus_intorder=3)
gfdir = GridFunction(fesH1)
gfdir.Interpolate (u0)

dense: 3080 x 2815
error 1.60475e-09


Assemble low-rank approximation single layer potential matrix (H-matrix) and solve for the Neumann data $u_1$ with iterative solver:

In [9]:
V = BilinearForm(fesL2, nonassemble=True)
with TaskManager(pajetrace=1000*1000*1000):
    SingleLayerPotentialOperator(V, intorder=10, leafsize=40, eta=3., eps=1e-10, method="svd")
    gfv = GridFunction(fesL2)
    CG(mat = V.mat, rhs = K.mat*gfdir.vec, pre=None, sol=gfv.vec, tol=1e-8, maxsteps=200, initialize=False, printrates=True)
#    solvers.BVP(bf=V, lf=K.mat*gfdir.vec, gf=gfv, solver=CGSolver)
Draw (-gfv, mesh, draw_vol=False);

HMatrix done 
dense: 3080 x 3080
error 4.86664e-09


AssertionError: 

In [ ]:
for t in Timers():
    if "ngbem" in t["name"]:
        print (t)

In [ ]:
gradu0 = CF( (u0.Diff(x), u0.Diff(y), u0.Diff(z)) )
n = specialcf.normal(3)
u1 = gradu0*n
Draw (u1, mesh, draw_vol=False);